In [58]:
import sqlite3
from sklearn.preprocessing import OneHotEncoder
import numpy as np
import numpy as np
import sqlite3
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.model_selection import train_test_split
from SQLiteDB import SQLiteDB
# 中文队名映射
team_name_mapping = {
    '老鹰': '亚特兰大老鹰',
    '凯尔特人': '波士顿凯尔特人',
    '篮网': '布鲁克林篮网',
    '黄蜂': '夏洛特黄蜂',
    '公牛': '芝加哥公牛',
    '骑士': '克利夫兰骑士',
    '独行侠': '达拉斯独行侠',
    '掘金': '丹佛掘金',
    '活塞': '底特律活塞',
    '勇士': '金州勇士',
    '火箭': '休斯顿火箭',
    '步行者': '印第安纳步行者',
    '快船': '洛杉矶快船',
    '湖人': '洛杉矶湖人',
    '灰熊': '孟菲斯灰熊',
    '热火': '迈阿密热火',
    '雄鹿': '密尔沃基雄鹿',
    '森林狼': '明尼苏达森林狼',
    '鹈鹕': '新奥尔良鹈鹕',
    '尼克斯': '纽约尼克斯',
    '雷霆': '俄克拉荷马城雷霆',
    '魔术': '奥兰多魔术',
    '76人': '费城76人',
    '太阳': '菲尼克斯太阳',
    '开拓者': '波特兰开拓者',
    '国王': '萨克拉门托国王',
    '马刺': '圣安东尼奥马刺',
    '猛龙': '多伦多猛龙',
    '爵士': '犹他爵士',
    '奇才': '华盛顿奇才'
}

# 连接 SQLite 数据库
conn = sqlite3.connect('C:\\code\\Project\\nba_springbootbatis\\springbootbatis\\src\\main\\resources\\nba_total.db')
cursor = conn.cursor()

# 查询比赛数据
query = '''
    SELECT leftName, leftGoal, rightName, rightGoal
    FROM nba_schedule
    WHERE leftGoal != 0 AND rightGoal != 0
      AND leftName !='' AND rightName !='' 
      AND leftName != '中国男篮' AND rightName != '中国男篮'
    ORDER BY date DESC
'''
games = cursor.execute(query).fetchall()

for game in games:
    print(game)

('灰熊', 118, '热火', 120)
('快船', 98, '灰熊', 99)
('魔术', 100, '森林狼', 115)
('勇士', 99, '热火', 102)
('76人', 103, '凯尔特人', 98)
('黄蜂', 97, '篮网', 90)
('猛龙', 89, '马刺', 100)
('活塞', 87, '爵士', 97)
('火箭', 95, '开拓者', 105)
('太阳', 77, '国王', 87)
('雄鹿', 79, '奇才', 91)
('步行者', 93, '骑士', 100)
('尼克斯', 90, '老鹰', 82)
('公牛', 81, '湖人', 107)
('鹈鹕', 82, '掘金', 91)
('雷霆', 88, '独行侠', 79)
('雄鹿', 90, '太阳', 115)
('独行侠', 101, '凯尔特人', 90)
('老鹰', 99, '公牛', 103)
('尼克斯', 91, '活塞', 90)
('76人', 96, '马刺', 80)
('猛龙', 73, '热火', 109)
('勇士', 90, '雷霆', 83)
('开拓者', 68, '黄蜂', 84)
('步行者', 71, '掘金', 86)
('灰熊', 88, '鹈鹕', 77)
('奇才', 73, '国王', 69)
('魔术', 100, '篮网', 102)
('湖人', 93, '骑士', 89)
('快船', 105, '爵士', 88)
('森林狼', 93, '火箭', 83)
('热火', 92, '独行侠', 79)
('国王', 105, '尼克斯', 106)
('猛龙', 76, '爵士', 86)
('凯尔特人', 89, '黄蜂', 84)
('灰熊', 104, '魔术', 98)
('太阳', 100, '雷霆', 99)
('湖人', 87, '老鹰', 86)
('勇士', 96, '骑士', 85)
('步行者', 98, '太阳', 94)
('尼克斯', 85, '篮网', 92)
('黄蜂', 80, '掘金', 66)
('公牛', 77, '活塞', 85)
('76人', 92, '森林狼', 90)
('奇才', 80, '开拓者', 82)
('快船', 11

In [78]:
import numpy as np
from sklearn.preprocessing import OneHotEncoder, StandardScaler
import sqlite3

# 定义我们需要的数值特征的索引（假设所有的数值列都要保留）
features_index = slice(2, -1)

# OneHotEncoder 用于处理文本数据（球员名字和队伍名称）
onehot_encoder = OneHotEncoder(sparse_output=False)

# 数据标准化
scaler = StandardScaler()

# 连接到 SQLite 数据库
db_path = 'C:\\code\\Project\\nba_springbootbatis\\springbootbatis\\src\\main\\resources\\nba_total.db'
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

# 查询需要的数据
query = '''
    SELECT leftName, leftGoal, rightName, rightGoal
    FROM nba_schedule
    WHERE leftGoal != 0 AND rightGoal != 0
      AND leftName != '' AND rightName != ''
      AND leftName != '中国男篮' AND rightName != '中国男篮'
    ORDER BY date DESC
'''
games = cursor.execute(query).fetchall()

# 准备存放 X 和 y 的列表
X = []
y = []

# 准备用于 OneHot 编码的文本数据（如球员名字和队伍名）
text_data = []

# 遍历每场比赛，使用映射找到 team_id，并查询 players_info 表中的数据
for game in games:
    left_name, left_goal, right_name, right_goal = game

    # 查询左队的 team_id
    cursor.execute("SELECT team_id FROM team_names WHERE team_name = ?", (left_name,))
    left_team_id = cursor.fetchone()

    # 查询右队的 team_id
    cursor.execute("SELECT team_id FROM team_names WHERE team_name = ?", (right_name,))
    right_team_id = cursor.fetchone()

    # 如果找到 team_id，则查询 players_info 表中的数据
    if left_team_id and right_team_id:
        # 查询左队的球员信息，并将空值替换为 0
        cursor.execute("SELECT * FROM players_info WHERE team_id = ?", (left_team_id[0],))
        left_team_players = cursor.fetchall()
        left_team_players = [[0 if val is None else val for val in player] for player in left_team_players]

        # 查询右队的球员信息，并将空值替换为 0
        cursor.execute("SELECT * FROM players_info WHERE team_id = ?", (right_team_id[0],))
        right_team_players = cursor.fetchall()
        right_team_players = [[0 if val is None else val for val in player] for player in right_team_players]

        # 提取左队和右队的数值特征
        left_team_data = np.array([player[features_index] for player in left_team_players])
        right_team_data = np.array([player[features_index] for player in right_team_players])

        # 提取左队和右队的文本特征（如球员名字）
        left_team_names = [player[1] for player in left_team_players]  # 假设 player[1] 是球员名字
        right_team_names = [player[1] for player in right_team_players]

        # 将文本特征添加到文本数据列表中，稍后用于 OneHot 编码
        text_data.append(left_team_names + right_team_names)

        # 合并左右队的数值数据
        combined_team_data = np.hstack((left_team_data.flatten(), right_team_data.flatten()))
        print('test',combined_team_data)
        X.append(combined_team_data)

        # 将 leftGoal 和 rightGoal 作为 y 的输出
        y.append([left_goal, right_goal])

# 转换 X 和 y 为 NumPy 数组
X = np.array(X)
y = np.array(y)

# 对文本特征进行 OneHot 编码
if len(text_data) > 0:
    flat_text_data = np.array([item for sublist in text_data for item in sublist]).reshape(-1, 1)
    onehot_encoded_text = onehot_encoder.fit_transform(flat_text_data)

    # 将 OneHot 编码结果与数值特征合并
    X_combined = np.hstack((X, onehot_encoded_text))

    # 对 X_combined 进行标准化处理
    X_scaled = scaler.fit_transform(X_combined)

    # 输出数据维度信息
    print(f"输入数据 X 的维度: {X_scaled.shape}")
    print(f"输出数据 y 的维度: {y.shape}")
else:
    print("没有可用的文本数据进行 OneHot 编码，检查数据来源。")

# 断开数据库连接
conn.close()


没有可用的文本数据进行 OneHot 编码，检查数据来源。


In [70]:
X

[]

In [ ]:
# 将 X 和 y 转换为 NumPy 数组
X = np.array(X)
y = np.array(y)

# 对 X 进行标准化处理
X_scaled = scaler.fit_transform(X)

# 输出数据维度信息
print(f"输入数据 X 的维度: {X_scaled.shape}")
print(f"输出数据 y 的维度: {y.shape}")

# 断开数据库连接


In [2]:
from sklearn.preprocessing import StandardScaler

# 对 X 进行标准化
scaler_X = StandardScaler()
X_scaled = scaler_X.fit_transform(X)

# 对 y 进行标准化
scaler_y = StandardScaler()
y_scaled = scaler_y.fit_transform(y)

# 切分训练集和测试集
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_scaled, test_size=0.2, random_state=42)

# 构建神经网络模型
model = Sequential()

# 第一层：输入层到第一个隐藏层，带有 10 个神经元，激活函数为 ReLU
model.add(Dense(10, input_dim=X_train.shape[1], activation='relu'))

# 第二层：第一个隐藏层到第二个隐藏层，带有 5 个神经元，激活函数为 ReLU
model.add(Dense(5, activation='relu'))

# 第三层：第二个隐藏层到输出层，带有 2 个神经元，用于回归任务（输出为连续值）
model.add(Dense(2))

# 编译模型，使用均方误差 (MSE) 作为损失函数，Adam 作为优化器
model.compile(optimizer='adam', loss='mse')

# 训练模型
model.fit(X_train, y_train, epochs=10, batch_size=6, validation_data=(X_test, y_test))

# 进行预测
y_pred_scaled = model.predict(X_test)

# 将预测值反标准化，恢复到原始的尺度
y_pred = scaler_y.inverse_transform(y_pred_scaled)

# 输出预测结果
print("预测结果：", y_pred[:5])  # 打印前5个预测结果


Epoch 1/10


c:\anaconda\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


12/12 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - loss: 1.3155 - val_loss: 1.3980
Epoch 2/10
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1.3219 - val_loss: 1.3478
Epoch 3/10
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1.2989 - val_loss: 1.3040
Epoch 4/10
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1.2452 - val_loss: 1.2732
Epoch 5/10
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1.0787 - val_loss: 1.2492
Epoch 6/10
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.9899 - val_loss: 1.2279
Epoch 7/10
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.8852 - val_loss: 1.1993
Epoch 8/10
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.9619 - val_loss: 1.1836
Epoch 9/10
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.9215 - val_loss: 1.1754
Epoch 10/10
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.8110 - val_loss: 1.1632
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
预测结果： [[82.8884  91.62527]
 [89.70219 83.62048]
 [90.3509  96.37128]
 [89.82402 89.80915]
 [87.57904 86.39254]]


In [21]:
model.summary()

Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_6 (Dense)                 │ (None, 2)              │           130 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 392 (1.54 KB)

 Trainable params: 130 (520.00 B)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 262 (1.03 KB)